In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661778 sha256=1ff231ccccf6f805e1b403f9df142dfe99ae5c04d1aa105ae58b383fb4ffdf3b
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00


In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-77llow0u/kobert-tokenizer_a55ad5f83a114dea92d8ad1f89df07e0
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-77llow0u/kobert-tokenizer_a55ad5f83a114dea92d8ad1f89df07e0
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=ab55e303914eb671a9e09235b0942c0afd864a7145e27fe937ef33e1e49598d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-wrnvtnfl/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
# GPU 사용 시
device = torch.device("cuda:0")

In [7]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content.drive')

Mounted at /content.drive


In [8]:
from sklearn.model_selection import train_test_split



In [9]:
df=pd.read_csv('/content/drive/MyDrive/project/text_cleaned_4.csv',encoding='UTF-8')

In [10]:
df.head(5)

,Unnamed: 0,text,mbti_ie,mbti_ns,mbti_tf,mbti_pj
0,0,궁금해요,NaN,NaN,NaN,NaN
1,1,사귄 지 얼마 안 됐고 섬 탈 때도 그냥 용건만 연락하는 편이길래 그러려니 했는데 ...,NaN,NaN,NaN,NaN
2,2,알려줘야 된다고 생각하세요? 아니면 굳이 필요 없다고 생각하세요? 자잘한 일상 공유...,NaN,NaN,NaN,NaN
3,3,애칭 어떻게 부르셨는지 알려주세요,i,n,t,j
4,4,나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해...,i,n,f,p


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62132 entries, 0 to 62131
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  62132 non-null  int64 
 1   text        62132 non-null  object
 2   mbti_ie     53366 non-null  object
 3   mbti_ns     53366 non-null  object
 4   mbti_tf     53366 non-null  object
 5   mbti_pj     53366 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.8+ MB


In [12]:
df['text'].count()

62132

In [13]:
df1=df.dropna(thresh=4)

In [14]:
df1['text'].count()

53366

In [15]:
df1['mbti_ie'].unique()

array(['i', 'e'], dtype=object)

In [16]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df1['mbti_ie'] = encoder.fit_transform(df1['mbti_ie'].values)

<ipython-input-16-e35b7333e5a2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['mbti_ie'] = encoder.fit_transform(df1['mbti_ie'].values)


In [17]:
df1['mbti_ie'].unique()

array([1, 0])

In [18]:
# [발화문, 상황] data_list 생성
data_list = []
for ques, label in zip (df1['text'], df1['mbti_ie']):
  data = []
  data.append(ques)
  data.append(str(label))

  data_list.append(data)

In [19]:
print(data)
print(data_list[:10])

['저도 그래요 ㅎㅎ   ', '0']
[['애칭 어떻게 부르셨는지 알려주세요 ', '1'], ['나만 남자친구랑 데이트 생각하는 거 같고 어찌나 서운하던지 근데 또 집에 가면 편해하고 너무 좋아하는 게 느껴지니까 편하기 쉴 생각하면 저도 그건 또 좋더라고요   표현방식도 다르고 연애 스타일도 달라서 그런지 남자친구는 해맑게 편하기만 한거 같고 저만 맘 고생하는 거 같고 그래요 ㅎㅎㅎ 남자친구가 예전 같지 않은 마음인지 원래 연애 스타일이 그런 건지 시간이 지나면 서로 맞춰져갈 부분인지 혼자 생각이 많아져서 답답한 마음에 얘기해 봅니다 ㅎㅎ ', '1'], ['istj와 isfj는 알파벳 한 끗 차이인데 istj이시면서 t와 f가 거의 비슷한 퍼센티지로 나오시는 분들...  전형적인 이티제와 어떤 점이 다를까요?  결이 좀 다른가요?? ', '0'], ['잇디 제분들 직장에서 수다떨기 좋아하시나요? ', '1'], ['estj 남 entj여 인 분들 연애 어떻게 하시고 계시나요? 사귄 지 얼마 안 됐는데 뭐를 중요하게 생각할지 어떻게 해야 할지 여쭤봅니다 ', '0'], ['대화를 해보면 말수가 적고 묻는 말에 대답만 함  굉장히 방어적인 태도를 취하며 본인도 그걸 알고 있음 +  사람에 따라 편차가 있으니 실제와는 다를 수 있습니다 재미로 읽으셔요  ', '0'], ['할 말 없다 싶으면?  그냥 하는 말 같고 그렇거든요......ㅋㅋ 제가 돌려서 말한다거나 빈말을 잘 못하는 편이여서 그런지(사회생활 빼고) estj 빈말 잘 하는 성향인가요? ', '0'], ['estj분들 거리 두고 싶거나 싫은 사람 혹은 정떨어진 사람과 연락할 때 하는 행동이 있을까요? ex)안읽씹, 12시간 뒤 답장 등등 ', '1'], ['estj 분들 중에서도 혼자일 때가 가장 빛이 나는 거 같고 편안하고 행복하신가요  ', '0'], ['막상 전해주려고 하니까 사귄 지 얼마 안 됐는데 부담스러워하지 않나 하는 생각이 들어서요 실용성 없는 선물도 별로 안 좋아한다고 들었고....  

In [20]:
X_train, X_test = train_test_split(data_list, test_size=0.2, random_state=123)

In [21]:
print(len(X_train), len(X_test))

42692 10674


In [22]:
tok = tokenizer.tokenize

In [23]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
# 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab = vocab, pad = pad, pair = pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))
        return (len(self.labels))

In [24]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [25]:
data_train = BERTDataset(X_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(X_test, 0, 1, tok, vocab, max_len, True, False)

In [26]:
data_train[0]

(array([   2, 3251, 4971, 2355, 6586,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(6, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [27]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [28]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [29]:
model = BERTClassifier(bertmodel, dr_rate = 0.5).to(device)

In [30]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [31]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [32]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [33]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-33-e6a38b13095b>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/668 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 0.7136585116386414 train acc 0.5625
epoch 1 batch id 201 loss 0.6503161787986755 train acc 0.5935945273631841
epoch 1 batch id 401 loss 0.6320370435714722 train acc 0.6107777431421446
epoch 1 batch id 601 loss 0.6322206854820251 train acc 0.6137167221297837
epoch 1 train acc 0.6151291167664671


<ipython-input-33-e6a38b13095b>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 4 test acc 0.6243675149700598


  0%|          | 0/668 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.4771912097930908 train acc 0.78125
epoch 5 batch id 201 loss 0.5026465654373169 train acc 0.7583955223880597
epoch 5 batch id 401 loss 0.39224672317504883 train acc 0.7772365960099751
epoch 5 batch id 601 loss 0.46447691321372986 train acc 0.7741524542429284
epoch 5 train acc 0.7745134730538922


  0%|          | 0/167 [00:00<?, ?it/s]

epoch 5 test acc 0.621186377245509


In [35]:
## 학습 모델 저장
PATH = '/content/drive/MyDrive/project' # google 드라이브 연동 해야함. 관련코드는 뺐음
torch.save(model, PATH + 'KoBERT_ie.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능